# Автоматизация

# Проект "Анализ взаимодействия пользователей с карточками Яндекс.Дзен"

1. [Описание проекта](#intro)
2. [Дашборд](#dashboard)   
3. [Презентация](#presentation) 

<a id="intro"></a>

## Описание проекта 

В сервисе Яндекс.Дзен пользователь постоянно взаимодействует с карточками. Каждую карточку определяют её тема и источник (у него тоже есть тема). Примеры тем: «Красота и здоровье», «Россия», «Путешествия». Пользователей системы характеризует возрастная категория (нарпимер, «26-30» или «45+»).

Есть три способа взаимодействия пользователей с системой:

    - Карточка отображена для пользователя (show);
    - Пользователь кликнул на карточку (click);
    - Пользователь просмотрел статью карточки (view).

Дата-инженеры разработали пайплайн, который берет данные из таблицы, в которых хранятся сырые данные, трансформирует данные и укладывает их в агрегирующую таблицу.

**Цель проекта:** Построение дашборда для анализа взаимодействия пользователей с карточками Яндекс.Дзен

**Техническое задание**

1. Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;
2. Кто будет основным пользователем дашборда: менеджеры по анализу контента;
3. Состав данных для дашборда:
    - История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
    - Разбивка событий по темам источников;
    - Таблица соответствия тем источников темам карточек;
4. По каким параметрам данные должны группироваться:
    - Дата и время;
    - Тема карточки;
    - Тема источника;
    - Возрастная группа;
5. Характер данных:
    - История событий по темам карточек — абсолютные величины с разбивкой по минутам;
    - Разбивка событий по темам источников — относительные величины (% событий);
    - Соответствия тем источников темам карточек - абсолютные величины;
6. Важность: все графики имеют равную важность;
7. Источники данных для дашборда: cырые данные о событиях взаимодействия пользователей с карточками (таблица log_raw);
8. База данных, в которой будут храниться агрегированные данные: дополнительные агрегированные таблицы в БД zen;
9. Частота обновления данных: один раз в сутки, в полночь по UTC.

**Задачи**

1. Загрузить данные в среде python
2. Создать дашборд в среде Tableau
3. Подготовить презентацию созданного дашборда

<a id="data"></a>

## Загрузка и предобработка данных 

In [1]:
# Загрузим данные в проект

# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# Загрузим данные из базы данных SQL в python
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string) 

query = ''' SELECT *
            FROM dash_visits
        '''

dash_visits = pd.io.sql.read_sql(query, con = engine) 

Проверим полноту и корректность данных

In [8]:
dash_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


In [17]:
# Проверка данных на дубликаты и пропуски
print('Явных дубликатов в базе данных:', dash_visits.duplicated().sum())
print('---')
print('Пропущенных значений в базе данных')
print(dash_visits.isna().sum())
print('---')
print('Уникальные значения поля item_topic')
print(dash_visits['item_topic'].unique())
print('---')
print('Уникальные значения поля source_topic')
print(dash_visits['source_topic'].unique())
print('---')
print('Уникальные значения поля age_segment')
print(dash_visits['age_segment'].unique())
print('---')
print(f"Выгрузка данных осуществлена за период с {dash_visits['dt'].min()} по {dash_visits['dt'].max()}")

Явных дубликатов в базе данных: 0
---
Пропущенных значений в базе данных
record_id       0
item_topic      0
source_topic    0
age_segment     0
dt              0
visits          0
dtype: int64
---
Уникальные значения поля item_topic
['Деньги' 'Дети' 'Женская психология' 'Женщины' 'Здоровье' 'Знаменитости'
 'Интересные факты' 'Искусство' 'История' 'Красота' 'Культура' 'Наука'
 'Общество' 'Отношения' 'Подборки' 'Полезные советы' 'Психология'
 'Путешествия' 'Рассказы' 'Россия' 'Семья' 'Скандалы' 'Туризм' 'Шоу'
 'Юмор']
---
Уникальные значения поля source_topic
['Авто' 'Деньги' 'Дети' 'Еда' 'Здоровье' 'Знаменитости' 'Интерьеры'
 'Искусство' 'История' 'Кино' 'Музыка' 'Одежда' 'Полезные советы'
 'Политика' 'Психология' 'Путешествия' 'Ремонт' 'Россия' 'Сад и дача'
 'Сделай сам' 'Семейные отношения' 'Семья' 'Спорт' 'Строительство'
 'Технологии' 'Финансы']
---
Уникальные значения поля age_segment
['18-25' '26-30' '31-35' '36-40' '41-45' '45+']
---
Выгрузка данных осуществлена за период с 2019-

**Вывод:** Данные представлены в корректном виде, без явных и скрытых дубликатов и пропусков значений

In [16]:
# Выгрузка даных в файл
dash_visits.to_csv('dash_visits.csv')

<a id="dashboard"></a>

## Дашборд

Дашборд выполнен в среде Tableau Public и доступен по ссылке

https://public.tableau.com/views/dash_visits_16613528794840/Dashboard1?:language=en-US&:display_count=n&:origin=viz_share_link![%D0%B8%D0%B7%D0%BE%D0%B1%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5.png](attachment:%D0%B8%D0%B7%D0%BE%D0%B1%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5.png)

<a id="presentation"></a>

## Презентация

Презентация дашборда доступна по ссылке

https://disk.yandex.ru/i/nBiVkkds5ETyzQ